# WRC Live Timing API Scraper

Simple exploration of grabbing data from WRC Live Timing API.

TO DO - how do we lookup `championshipId`?

NOTE: THIS MAY BE BROKEN AND DEPRECATED — SHINY IS BEST LATEST

In [1]:
from urllib.parse import urljoin
from parse import parse
import pathlib

import pandas as pd
import requests

#%pip install requests-cache
import requests_cache
requests_cache.install_cache("demo_cache3")

In [2]:
CURRENT_YEAR = 2025

WRC_API_BASE = "https://api.wrc.com/content/{path}"

In [68]:
from itertools import zip_longest


def _WRC_json(path, base=WRC_API_BASE, retUrl=False):
    """Return JSON from API."""
    url = urljoin(base, path)
    if retUrl:
        return url
    # print(f"Fetching: {url}")
    r = requests.get(url)
    rj = r.json()
    if "status" in rj and rj["status"]=="Not Found":
        return {}
    return r.json()


def convert_date_range(date_range_str):
    """Convert date of from `19 - 22 JAN 2023` to date range."""
    r = parse("{start_day} - {end_day} {month} {year}", date_range_str)
    start_date = pd.to_datetime(
        f"{r['start_day']} {r['month']} {r['year']}", format="%d %b %Y"
    )
    end_date = pd.to_datetime(
        f"{r['end_day']} {r['month']} {r['year']}", format="%d %b %Y"
    )
    return pd.date_range(start=start_date, end=end_date)


def timeify(df, col, typ=None):
    """Convert a column  to a datetime inplace."""
    if typ == "daterange":
        df[col] = df[col].apply(convert_date_range)
    else:
        df[col] = pd.to_datetime(df[col].astype(int), unit="ms")


def tablify(json_data, subcolkey=None, addcols=None):
    """Generate table from separate colnames/values JSON."""
    # Note that the JSON may be a few rows short cf. provided keys
    if subcolkey is None:
        results = []
        for values in json_data["values"]:
            result_dict = {}
            # Zip keys and values, filling None for missing values
            zipped_values = zip_longest(json_data["fields"], values, fillvalue=None)
            # Convert the zipped values to a dictionary and update the result_dict
            result_dict.update(dict(zipped_values))
            results.append(result_dict)
        return pd.DataFrame(results)
    else:
        df = pd.DataFrame(columns=json_data["fields"])
        for value in json_data["values"]:
            _df = pd.DataFrame(value[subcolkey])
            if len(_df.columns) < len(json_data["fields"]):
                _df[[json_data["fields"][len(_df.columns) :]]] = None
            _df.columns = json_data["fields"]
            if addcols:
                for c in addcols:
                    _df[c] = value[c]
            for c in [k for k in value.keys() if k != subcolkey]:
                _df[c] = value[c]
                df = pd.concat([df, _df])
        return df

def stage_id_annotations(df, eventId=None, rallyId=None, stageId=None):
    if "eventId" not in df.columns:
        df["eventId"] = eventId
    if "rallyId" not in df.columns:
        df["rallyId"] = rallyId
    if "stageId" not in df.columns:
        df["stageId"] = stageId

## Season Calendar

The API call `https://api.wrc.com/content/filters/calendar` gives a list of current or latest (last, if season's end) events, from which we can get a season identifer. For the 2023 WRC championship, the season ID is `20`

We can get the championship IDs from the WRC live timing results pages. We could scrpe this but it requires browser automation — the page is not loaded as simple HTML, but created from chunks with obscure identifiers pulled using Javascript.

In [66]:
# Data from rendered page:
#https://www.wrc.com/live-timing?liveTimingMenu=overall_livetiming

import re

category_html = """
<div data-key="287" class="sc-dVhcbM hzWUOI" active="true">ALL</div>
<div data-key="289" class="sc-dVhcbM hzWUOI">WRC</div>
<div data-key="291" class="sc-dVhcbM hzWUOI">WRC2</div>
<div data-key="298" class="sc-dVhcbM hzWUOI">WRC3</div>
<div data-key="300" class="sc-dVhcbM hzWUOI">Junior WRC</div>
<div data-key="293" class="sc-dVhcbM hzWUOI">WRC2 Challenger</div>
<div data-key="296" class="sc-dVhcbM hzWUOI">Master Cup</div>
"""


category_map = {'ALL': 'all',
 'WRC': 'wrc',
 'WRC2': 'wrc2',
 'WRC3': 'wrc3',
 'Junior WRC': 'jwrc',
 'WRC2 Challenger': 'wrc2c',
 'Master Cup': 'mcup'}

category_map2 = {value: key for key, value in category_map.items()}

championshipIds = {}

pattern = r'.*data-key="(\d+)"[^>]*>([^<]+)<\/div>'
for c in category_html.split("\n"):
    if not c:
        continue

    match = re.search(pattern, c)
    if match:
        code = match.group(1)      # Captured data-key value
        category = match.group(2)  # Captured category text
        championshipIds[category_map[category]] = int(code)

"""
championshipIds = {'all': 287,
                   'wrc': 289,
                   'wrc2': 291,
                   'wrc3': 298,
                   'jwrc': 300,
                   'wrc2c': 293,
                   'mcup': 296}
 """
championshipIds

{'all': 287,
 'wrc': 289,
 'wrc2': 291,
 'wrc3': 298,
 'jwrc': 300,
 'wrc2c': 293,
 'mcup': 296}

In [30]:
championshipId = championshipIds["wrc"]

In [32]:
stub = f"filters/calendar?language=en&size=20&championship=wrc&origin=vcms&year={CURRENT_YEAR}"
json_data = _WRC_json(stub)
keys_to_keep = ['id', 'guid', 'title', 'location', 'startDate', 'endDate', 'eventId',
       'rallyId', 'description', 'round', 'cvpSeriesLink', 'sponsor', 'images',
       'season', 'competition', 'country', 'asset', '__typename', 'type',
       'uid', 'seriesUid', 'releaseYear', 'availableOn', 'availableTill',
       'startDateLocal', 'endDateLocal', 'finishDate', 'championship',
       'championshipLogo'
]
filtered_data = [
    {key: item[key] for key in keys_to_keep if key in item}
    for item in json_data["content"]
]

pd.DataFrame(filtered_data)

,id,guid,title,location,startDate,endDate,eventId,rallyId,description,round,...,uid,seriesUid,releaseYear,availableOn,availableTill,startDateLocal,endDateLocal,finishDate,championship,championshipLogo
0,K1Lg,WRC_2025_01,Rallye Monte-Carlo,Monaco,1737558000000,1737898200000,534,582,The most unpredictable rally of the year. Rela...,1,...,K1Lg,WRC_2025_01,2025,1737558000000,1737898200000,2025-01-22T16:00:00+01:00,2025-01-26T14:30:00+01:00,1737898200000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
1,skC3,WRC_2025_02,WRC Rally Sweden,sweden,1739433600000,1739718000000,535,583,The WRC’s ultimate winter challenge. Watch in ...,2,...,skC3,WRC_2025_02,2025,1739433600000,1739718000000,2025-02-13T09:00:00+01:00,2025-02-16T16:00:00+01:00,1739718000000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
2,G2NQ,WRC_2025_03,WRC Safari Rally Kenya,"Naivasha, Kenya",1742450400000,1742734800000,536,584,Held on the untamed terrains of Africa’s breat...,3,...,G2NQ,WRC_2025_03,2025,1742450400000,1742734800000,2025-03-20T09:00:00+03:00,2025-03-23T16:00:00+03:00,1742734800000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
3,zhuM,WRC_2025_04,WRC Rally Islas Canarias,"Islas Canarias, Spain",1745478000000,1745762400000,538,586,Well-known to FIA European Rally Championship ...,4,...,zhuM,WRC_2025_04,2025,1745478000000,1745762400000,2025-04-24T09:00:00+02:00,2025-04-27T16:00:00+02:00,1745762400000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
4,edGQ,WRC_2025_05,WRC Vodafone Rally de Portugal,"Matosinhos, Porto",1747296000000,1747580400000,540,588,Fast but technical gravel roads inland from Po...,5,...,edGQ,WRC_2025_05,2025,1747296000000,1747580400000,2025-05-15T09:00:00+01:00,2025-05-18T16:00:00+01:00,1747580400000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
5,k5hf,WRC_2025_06,WRC Rally Italia Sardegna,"Olbia, Sardinia",1749106800000,1749391200000,542,590,Don’t let the picturesque Mediterranean backdr...,6,...,k5hf,WRC_2025_06,2025,1749106800000,1749391200000,2025-06-05T09:00:00+02:00,2025-06-08T16:00:00+02:00,1749391200000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
6,eaYx,WRC_2025_07,WRC EKO Acropolis Rally Greece,Lamia,1750917600000,1751202000000,544,592,A legendary WRC fixture. Twisty gravel mountai...,7,...,eaYx,WRC_2025_07,2025,1750917600000,1751202000000,2025-06-26T09:00:00+03:00,2025-06-29T16:00:00+03:00,1751202000000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
7,aIpJ,WRC_2025_08,WRC Delfi Rally Estonia,Estonia,1752732000000,1753016400000,546,594,Back on the WRC Calendar! With its' super fast...,8,...,aIpJ,WRC_2025_08,2025,1752732000000,1753016400000,2025-07-17T09:00:00+03:00,2025-07-20T16:00:00+03:00,1753016400000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
8,CaGI,WRC_2025_09,WRC Secto Rally Finland,Jyväskylä,1753941600000,1754226000000,547,595,A mecca for rally drivers and enthusiasts alik...,9,...,CaGI,WRC_2025_09,2025,1753941600000,1754226000000,2025-07-31T09:00:00+03:00,2025-08-03T16:00:00+03:00,1754226000000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
9,nMcx,WRC_2025_10,WRC Rally del Paraguay,Paraguay,1756386000000,1756670400000,549,597,Welcome to the 2025 WRC calendar! Rally del Pa...,10,...,nMcx,WRC_2025_10,2025,1756386000000,1756670400000,2025-08-28T09:00:00-04:00,2025-08-31T16:00:00-04:00,1756670400000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...


In [33]:
def getFullCalendar(year=CURRENT_YEAR, championship="wrc", size=20):
    stub = f"filters/calendar?language=en&size={size}&championship={championship}&origin=vcms&year={year}"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.dataFrame()
    # return json_data
    return pd.DataFrame(json_data["content"])


df_fullcal = getFullCalendar()
df_fullcal.head()

,id,guid,title,location,startDate,endDate,eventId,rallyId,description,round,...,uid,seriesUid,releaseYear,availableOn,availableTill,startDateLocal,endDateLocal,finishDate,championship,championshipLogo
0,K1Lg,WRC_2025_01,Rallye Monte-Carlo,Monaco,1737558000000,1737898200000,534,582,The most unpredictable rally of the year. Rela...,1,...,K1Lg,WRC_2025_01,2025,1737558000000,1737898200000,2025-01-22T16:00:00+01:00,2025-01-26T14:30:00+01:00,1737898200000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
1,skC3,WRC_2025_02,WRC Rally Sweden,sweden,1739433600000,1739718000000,535,583,The WRC’s ultimate winter challenge. Watch in ...,2,...,skC3,WRC_2025_02,2025,1739433600000,1739718000000,2025-02-13T09:00:00+01:00,2025-02-16T16:00:00+01:00,1739718000000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
2,G2NQ,WRC_2025_03,WRC Safari Rally Kenya,"Naivasha, Kenya",1742450400000,1742734800000,536,584,Held on the untamed terrains of Africa’s breat...,3,...,G2NQ,WRC_2025_03,2025,1742450400000,1742734800000,2025-03-20T09:00:00+03:00,2025-03-23T16:00:00+03:00,1742734800000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
3,zhuM,WRC_2025_04,WRC Rally Islas Canarias,"Islas Canarias, Spain",1745478000000,1745762400000,538,586,Well-known to FIA European Rally Championship ...,4,...,zhuM,WRC_2025_04,2025,1745478000000,1745762400000,2025-04-24T09:00:00+02:00,2025-04-27T16:00:00+02:00,1745762400000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
4,edGQ,WRC_2025_05,WRC Vodafone Rally de Portugal,"Matosinhos, Porto",1747296000000,1747580400000,540,588,Fast but technical gravel roads inland from Po...,5,...,edGQ,WRC_2025_05,2025,1747296000000,1747580400000,2025-05-15T09:00:00+01:00,2025-05-18T16:00:00+01:00,1747580400000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...


In [34]:
len(df_fullcal)

14

In [35]:
df_fullcal.columns

Index(['id', 'guid', 'title', 'location', 'startDate', 'endDate', 'eventId',
       'rallyId', 'description', 'round', 'cvpSeriesLink', 'sponsor', 'images',
       'season', 'competition', 'country', 'asset', '__typename', 'type',
       'uid', 'seriesUid', 'releaseYear', 'availableOn', 'availableTill',
       'startDateLocal', 'endDateLocal', 'finishDate', 'championship',
       'championshipLogo'],
      dtype='object')

In [36]:
df_fullcal

,id,guid,title,location,startDate,endDate,eventId,rallyId,description,round,...,uid,seriesUid,releaseYear,availableOn,availableTill,startDateLocal,endDateLocal,finishDate,championship,championshipLogo
0,K1Lg,WRC_2025_01,Rallye Monte-Carlo,Monaco,1737558000000,1737898200000,534,582,The most unpredictable rally of the year. Rela...,1,...,K1Lg,WRC_2025_01,2025,1737558000000,1737898200000,2025-01-22T16:00:00+01:00,2025-01-26T14:30:00+01:00,1737898200000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
1,skC3,WRC_2025_02,WRC Rally Sweden,sweden,1739433600000,1739718000000,535,583,The WRC’s ultimate winter challenge. Watch in ...,2,...,skC3,WRC_2025_02,2025,1739433600000,1739718000000,2025-02-13T09:00:00+01:00,2025-02-16T16:00:00+01:00,1739718000000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
2,G2NQ,WRC_2025_03,WRC Safari Rally Kenya,"Naivasha, Kenya",1742450400000,1742734800000,536,584,Held on the untamed terrains of Africa’s breat...,3,...,G2NQ,WRC_2025_03,2025,1742450400000,1742734800000,2025-03-20T09:00:00+03:00,2025-03-23T16:00:00+03:00,1742734800000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
3,zhuM,WRC_2025_04,WRC Rally Islas Canarias,"Islas Canarias, Spain",1745478000000,1745762400000,538,586,Well-known to FIA European Rally Championship ...,4,...,zhuM,WRC_2025_04,2025,1745478000000,1745762400000,2025-04-24T09:00:00+02:00,2025-04-27T16:00:00+02:00,1745762400000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
4,edGQ,WRC_2025_05,WRC Vodafone Rally de Portugal,"Matosinhos, Porto",1747296000000,1747580400000,540,588,Fast but technical gravel roads inland from Po...,5,...,edGQ,WRC_2025_05,2025,1747296000000,1747580400000,2025-05-15T09:00:00+01:00,2025-05-18T16:00:00+01:00,1747580400000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
5,k5hf,WRC_2025_06,WRC Rally Italia Sardegna,"Olbia, Sardinia",1749106800000,1749391200000,542,590,Don’t let the picturesque Mediterranean backdr...,6,...,k5hf,WRC_2025_06,2025,1749106800000,1749391200000,2025-06-05T09:00:00+02:00,2025-06-08T16:00:00+02:00,1749391200000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
6,eaYx,WRC_2025_07,WRC EKO Acropolis Rally Greece,Lamia,1750917600000,1751202000000,544,592,A legendary WRC fixture. Twisty gravel mountai...,7,...,eaYx,WRC_2025_07,2025,1750917600000,1751202000000,2025-06-26T09:00:00+03:00,2025-06-29T16:00:00+03:00,1751202000000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
7,aIpJ,WRC_2025_08,WRC Delfi Rally Estonia,Estonia,1752732000000,1753016400000,546,594,Back on the WRC Calendar! With its' super fast...,8,...,aIpJ,WRC_2025_08,2025,1752732000000,1753016400000,2025-07-17T09:00:00+03:00,2025-07-20T16:00:00+03:00,1753016400000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
8,CaGI,WRC_2025_09,WRC Secto Rally Finland,Jyväskylä,1753941600000,1754226000000,547,595,A mecca for rally drivers and enthusiasts alik...,9,...,CaGI,WRC_2025_09,2025,1753941600000,1754226000000,2025-07-31T09:00:00+03:00,2025-08-03T16:00:00+03:00,1754226000000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
9,nMcx,WRC_2025_10,WRC Rally del Paraguay,Paraguay,1756386000000,1756670400000,549,597,Welcome to the 2025 WRC calendar! Rally del Pa...,10,...,nMcx,WRC_2025_10,2025,1756386000000,1756670400000,2025-08-28T09:00:00-04:00,2025-08-31T16:00:00-04:00,1756670400000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...


Let's get the `seasonId` from a season record in the `season` column:

In [37]:
df_fullcal["season"][0]

{'id': '966f4272-df7f-41af-8686-7c5b8fd1dca1',
 'name': None,
 'seasonId': '34',
 'title': '2025',
 '__typename': 'Season'}

In [38]:
CURRENT_SEASON = df_fullcal["season"][0]["seasonId"]

seasonId = CURRENT_SEASON

# Check
# assert int(CURRENT_SEASON)==28 if CURRENT_YEAR==2024 else False
# And / or check the title?
# assert df_fullcal["season"][0]['title'] == '2024'

CURRENT_SEASON

'34'

In [39]:
# https://api.wrc.com/content/result/calendar?season=20&championship=wrc

def getResultsCalendar(seasonId=CURRENT_SEASON,  retUrl=False):
    """Get the WRC Calendar for a given season ID as a JSON result."""
    stub = f"result/calendar?season={seasonId}&championship=wrc"
    if retUrl:
        return stub
    json_data = _WRC_json(stub)
    df_calendar = tablify(json_data)
    #timeify(df_calendar, "date", "daterange")
    #timeify(df_calendar, "startDate")
    #timeify(df_calendar, "finishDate")
    # df_calendar.set_index("id", inplace=True)
    return df_calendar

In [40]:
getResultsCalendar().columns

Index(['id', 'rallyTitle', 'ROUND', 'rallyCountry', 'rallyCountryImage',
       'rallyId', 'date', 'startDate', 'finishDate', 'driverId',
       'driverCountryImage', 'driver', 'coDriverId', 'coDriverCountryImage',
       'coDriver', 'teamId', 'teamLogo', 'teamName', 'manufacturer'],
      dtype='object')

In [41]:
getResultsCalendar(retUrl=True)

'result/calendar?season=34&championship=wrc'

In [42]:
getResultsCalendar().head()

,id,rallyTitle,ROUND,rallyCountry,rallyCountryImage,rallyId,date,startDate,finishDate,driverId,driverCountryImage,driver,coDriverId,coDriverCountryImage,coDriver,teamId,teamLogo,teamName,manufacturer
0,K1Lg,Rallye Monte-Carlo,1,Monaco,Flags/MCO.png,582,22 - 26 JAN 2025,1737558000000,1737898200000,,,,,,,,,,
1,skC3,WRC Rally Sweden,2,Sweden,Flags/SWE.png,583,13 - 16 FEB 2025,1739433600000,1739718000000,,,,,,,,,,
2,G2NQ,WRC Safari Rally Kenya,3,Kenya,Flags/KEN.png,584,20 - 23 MAR 2025,1742450400000,1742734800000,,,,,,,,,,
3,zhuM,WRC Rally Islas Canarias,4,Spain,Flags/ESP.png,586,24 - 27 APR 2025,1745478000000,1745762400000,,,,,,,,,,
4,edGQ,WRC Vodafone Rally de Portugal,5,Portugal,Flags/PRT.png,588,15 - 18 MAY 2025,1747296000000,1747580400000,,,,,,,,,,


In [43]:
getResultsCalendar().tail()

,id,rallyTitle,ROUND,rallyCountry,rallyCountryImage,rallyId,date,startDate,finishDate,driverId,driverCountryImage,driver,coDriverId,coDriverCountryImage,coDriver,teamId,teamLogo,teamName,manufacturer
8,CaGI,WRC Secto Rally Finland,9,Finland,Flags/FIN.png,595,31 - 03 AUG 2025,1753941600000,1754226000000,,,,,,,,,,
9,nMcx,WRC Rally del Paraguay,10,Paraguay,Flags/PRY.png,597,28 - 31 AUG 2025,1756386000000,1756670400000,,,,,,,,,,
10,XpJ0,WRC Rally Chile Bio Bío,11,Chile,Flags/CHL.png,599,11 - 14 SEP 2025,1757592000000,1757876400000,,,,,,,,,,
11,nQmX,WRC Central European Rally,12,Europe,Flags/EUR.png,601,16 - 19 OCT 2025,1760599800000,1760880600000,,,,,,,,,,
12,CMXn,WRC FORUM8 Rally Japan,13,Japan,Flags/JPN.png,602,06 - 09 NOV 2025,1762387200000,1762671600000,,,,,,,,,,


In [44]:
import datetime

def timeNow(typ="ms"):
    now = int(datetime.datetime.now().timestamp())
    if typ=="ms":
        now *=1000
    return now

timeNow()

1737627373000

In [45]:
eventId, rallyId = df_fullcal[df_fullcal["startDate"] < timeNow()].iloc[-1][
    ["eventId", "rallyId"]
]
eventId, rallyId

('534', '582')

In [46]:
## Stages

def getStageDetails(eventId, rallyId):
    stub = f"result/stages?eventId={eventId}&rallyId={rallyId}&championship=wrc"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.dataFrame()
    df_stageDetails = tablify(json_data)
    return df_stageDetails


df_stageDetails = getStageDetails(eventId, rallyId)
df_stageDetails.head(3)

,id,STAGE,STAGE TYPE,stageId,eventId,STATUS,day,name,distance
0,ee55daa1-fd07-574f-835a-f39008081b95,SHD,shakedown,SHD,534,Running,,Shakedown,
1,5d822449-4aa8-578d-a4b5-11de9d6b5c8e,SS1,SpecialStage,8210,534,ToRun,Thursday,SS1 Digne-les-Bains - Chaudon-Norante 1 (19.01km),19.01
2,395add0b-a1ac-5202-bb72-b247dc5fe024,SS2,SpecialStage,8211,534,ToRun,Thursday,SS2 Faucon-du-Caire - Bréziers (21.18km),21.18


In [47]:
df_stageDetails.tail()

,id,STAGE,STAGE TYPE,stageId,eventId,STATUS,day,name,distance
15,b49dd66a-212c-5b82-b817-354f6fa67a4d,SS15,SpecialStage,8224,534,ToRun,Saturday,SS15 La-Bâtie-des-Fonts - Aspremont 2 (17.85km),17.85
16,caea0055-6df8-55a3-9ea4-5c7f6435461d,SS16,SpecialStage,8225,534,ToRun,Sunday,SS16 Avançon - Notre-Dame du Laus 2 (13.97km),13.97
17,b97e8a17-3ec8-5377-b9ac-3adada97ac1e,SS17,SpecialStage,8226,534,ToRun,Sunday,SS17 Digne-les-Bains - Chaudon-Norante 2 (19.0...,19.01
18,19513ad9-1929-5f77-a4c5-004587652565,SS18,PowerStage,8227,534,ToRun,Sunday,SS18 La Bollène-Vésubie - Peïra-Cava (17.92km),17.92
19,f8f13d04-bbaf-58d1-906d-c2d459999c1b,FINAL,overall result,FINAL,534,ToRun,,Final Result,


In [49]:
def getOverall(eventId, rallyId, stageId, championship="wrc", group="all"):
    stub = f"result/stageResult?eventId={eventId}&rallyId={rallyId}&stageId={stageId}&championshipId={championshipId}&championship={championship}"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.DataFrame()
    df_overall = tablify(json_data)
    stage_id_annotations(df_overall, eventId, rallyId, stageId)
    if group=="all":
        return df_overall
    else:
        return df_overall[df_overall["groupClass"]==group]

stageId = df_stageDetails[df_stageDetails["stageId"]!="FINAL"].iloc[-1]["stageId"]

getOverall(eventId, rallyId, stageId)#.head(3)

""


In [50]:
getOverall(eventId, rallyId, stageId).columns

RangeIndex(start=0, stop=0, step=1)

In [51]:
def getStageTimes(eventId, rallyId, stageId, championship="wrc"):
    stub = f"result/stageTimes?eventId={eventId}&rallyId={rallyId}&stageId={stageId}&championshipId={championshipId}&championship={championship}"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.DataFrame()
    df_stageTimes = tablify(json_data)
    stage_id_annotations(df_stageTimes, eventId, rallyId, stageId)
    return df_stageTimes


getStageTimes(eventId, rallyId, stageId).head(3)

""


In [166]:
#getStageTimes(365, 369, 4161).head(3)

In [52]:
def getSplitTimes(championshipId, eventId, rallyId, stageId, championship="wrc"):
    if CURRENT_YEAR>2023:
        stub = f"result/splitTime?championshipId={championshipId}&eventId={eventId}&rallyId={rallyId}&stageId={stageId}&championship={championship}"
    else:
        stub = f"result/splitTime?eventId={eventId}&rallyId={rallyId}&stageId={stageId}&championship={championship}"

    json_data = _WRC_json(stub)
    if not json_data:
        return pd.DataFrame()
    df_splitTimes = tablify(json_data)
    stage_id_annotations(df_splitTimes, eventId, rallyId, stageId)
    return df_splitTimes


getSplitTimes(championshipId, eventId, rallyId, stageId).head(3)

""


In [54]:
getSplitTimes(championshipId, eventId, rallyId, stageId).columns

RangeIndex(start=0, stop=0, step=1)

In [55]:
championshipId

'289'

In [56]:
def getStageWinners(eventId, championship="wrc"):
    championshipId = championshipIds[championship]
    stub = f"result/stageWinners?eventId={eventId}&championshipId={championshipId}"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.DataFrame()
    df_stageWinners = tablify(json_data)
    return df_stageWinners


getStageWinners(eventId, ).head(3)

,id,carNo,stageNo,stageName,stageType,stageId,eventId,entryId,driverId,driverCountry,...,coDriverId,coDriverCountry,coDriverCountryImage,coDriver,teamId,team/car,teamName,teamLogo,eligibility,time
0,534,,SS1,Digne-les-Bains - Chaudon-Norante 1 (19.01 km),SpecialStage,8210,ae7329c9-79b3-5d96-886c-22cca6217764,,,,...,,,,,,,,,,
1,534,,SS2,Faucon-du-Caire - Bréziers (21.18 km),SpecialStage,8211,ae7329c9-79b3-5d96-886c-22cca6217764,,,,...,,,,,,,,,,
2,534,,SS3,Avançon - Notre-Dame du Laus 1 (13.97 km),SpecialStage,8212,ae7329c9-79b3-5d96-886c-22cca6217764,,,,...,,,,,,,,,,


In [57]:
def getItinerary(eventId):
    stub = f"result/itinerary?eventId={eventId}&extended=true"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.DataFrame()
    df_itinerary = tablify(json_data, "values")
    return df_itinerary


getItinerary(eventId).tail(3)

,stage,eventId,stageId,distance,timingPrecision,firstCarDueDateTime,firstCarDueDateTimeMs,controlPenalties,status,type,location,targetDuration,targetDurationMs,id,order,date
12,SF18,534,19513ad9-1929-5f77-a4c5-004587652565,,Thousandth,,,None,ToRun,FlyingFinish,La Bollène-Vésubie - Peïra-Cava,null,null,1489.0,4.0,Sunday 26th January
13,TC18A,534,null,52.65 km,Minute,14:05,2025-01-26T14:05:00+01:00,Late,ToRun,TimeControl,Monaco - Technical Zone IN,01:50:00,6600000,1489.0,4.0,Sunday 26th January
14,TC18B,534,null,0.1 km,Minute,14:15,2025-01-26T14:15:00+01:00,Late,ToRun,TimeControl,Monaco - Technical Zone OUT - Parc Fermé IN,00:10:00,600000,1489.0,4.0,Sunday 26th January


In [70]:
def getStartlist(eventId):
    stub = f"result/startLists?eventId={eventId}"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.DataFrame()
    df_startlist = tablify(json_data, "startListItems", addcols=["date", "startDateTimeLocal"])
    return df_startlist


getStartlist(eventId).tail(3)

,order,startDateTimeLocal,carNo,driverId,driverCountry,driverCountryImage,driver,coDriverId,coDriverCountry,coDriverCountryImage,coDriver,teamId,team/car,teamName,teamLogo,eligibility,groupClass,priority,date,id
65,66,2025-01-23T14:30:00+01:00,#80,1bdf1a2f-c923-52e7-8ba7-d9f31ae61654,France,Flags/FRA.png,Lilian VIALLE,2456caf1-5123-55bb-8622-c2331d4dd695,France,Flags/FRA.png,Manuel GHIRARDELLO,a72b2006-2ec2-547a-b2ce-b97ac1bee818,Clio RS Line,Renault,teamLogo/renault.png,,RC5,None,Thursday 2025-01-23,724df7c0-6f51-5ff2-96f1-a610e3579961
66,67,2025-01-23T14:30:00+01:00,#81,5d405c6b-cf2d-52af-99cb-31628cf1f9e7,France,Flags/FRA.png,Christophe TRUCHET,5fec7431-1c85-59ce-8f13-1668881a7d07,France,Flags/FRA.png,Barbara TRUCHET,a72b2006-2ec2-547a-b2ce-b97ac1bee818,Clio RS Line,Renault,teamLogo/renault.png,,RC5,None,Thursday 2025-01-23,724df7c0-6f51-5ff2-96f1-a610e3579961
67,68,2025-01-23T14:30:00+01:00,#82,e3b7053c-d22f-59fc-857d-fd04170b10f4,France,Flags/FRA.png,Sébastien MATTEI,90ac6118-01bb-5a36-aa55-6b05b88dbaa7,France,Flags/FRA.png,Loan BIAGETTI,a72b2006-2ec2-547a-b2ce-b97ac1bee818,Clio RS Line,Renault,teamLogo/renault.png,,RC5,None,Thursday 2025-01-23,724df7c0-6f51-5ff2-96f1-a610e3579961


In [59]:
def getPenalties(eventId):
    stub = f"result/penalty?eventId={eventId}"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.DataFrame()
    df_penalties = tablify(json_data)
    return df_penalties


getPenalties(eventId).head(3)

""


In [60]:
def getRetirements(eventId):
    stub = f"result/retirements?eventId={eventId}"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.DataFrame()
    df_retirements = tablify(json_data)
    return df_retirements


getRetirements(eventId).head(3)

,id,carNo,driverId,driverCountry,driverCountryImage,driver,coDriverId,coDriverCountry,coDriverCountryImage,coDriver,...,team/car,teamName,teamLogo,eligibility,groupClass,rallyId,entryId,controlId,reason,control
0,8894,#32,68be047a-0d3d-5506-bdd8-4e0f27e9c6d3,France,Flags/FRA.png,Matthieu MARGAILLAN,64016eb1-5b31-56f6-9892-6944b44142f9,France,Flags/FRA.png,Mathilde MARGAILLAN,...,Fabia RS,Skoda,teamLogo/skoda.png,WRC2 (DC/CC),RC2,582,54690,34717,MECHANICAL,TC0


In [63]:
def getChampionship(
    seasonId, eventId, championship_type="driver", championship="all", retUrl=False
):
    """ championship_tpe: driver | codriver | manufacturer"""
    championshipId = championshipIds[championship]
    stub = f"result/championshipresult?seasonId={seasonId}&championshipId={championshipId}&type={championship_type}&championship={championship}"
    if retUrl:
        return stub
    json_data = _WRC_json(stub)
    if not json_data or "message" in json_data and "championship standing unavailble" in json_data["message"] :
        return pd.DataFrame()
    print(json_data)
    df_splitTimes = tablify(json_data)
    return df_splitTimes


championship_type = "driver"  # where is list?

getChampionship(seasonId, championship_type).head(3)

""


In [64]:
getChampionship(seasonId, eventId, championship_type)
#championshipresult?seasonId=28&championshipId=245&type=driver&championship=wrc

""


In [65]:
def time_to_seconds(time_str, retzero=False):
    if not time_str or not isinstance(time_str, str):
        return 0 if retzero else np.nan

    try:
        # Handle sign
        is_negative = time_str.startswith("-")
        time_str = time_str.lstrip("+-")

        # Split the time string into parts
        parts = time_str.split(":")

        # Depending on the number of parts, interpret hours, minutes, and seconds
        if len(parts) == 3:  # Hours, minutes, seconds.tenths
            hours, minutes, seconds = parts
            total_seconds = int(hours) * 3600 + int(minutes) * 60 + float(seconds)
        elif len(parts) == 2:  # Minutes, seconds.tenths
            minutes, seconds = parts
            total_seconds = int(minutes) * 60 + float(seconds)
        else:
            total_seconds = float(parts[0])

        # Apply negative sign if needed
        total_seconds = -total_seconds if is_negative else total_seconds

        # Round to 1 decimal place
        return round(total_seconds, 1)
    
    except (ValueError, TypeError):
        return 0 if retzero else np.nan

# Function to apply time delta
def apply_time_delta(base_time_str, delta_str):
    # Convert base time and delta time to seconds
    base_seconds = time_to_seconds(base_time_str)
    delta_seconds = time_to_seconds(delta_str)

    if base_seconds is None or delta_seconds is None:
        return None

    # If delta is positive, add, if negative, subtract
    if delta_str.startswith('-'):
        return round(base_seconds - delta_seconds, 1)
    else:
        return round(base_seconds + delta_seconds, 1)

In [178]:
import sqlite_utils
#!rm wrc_2024_archive_results.db
_db_filepath = f"wrc_{CURRENT_YEAR}_archive_results.db"
try:
    pathlib.Path.unlink(_db_filepath)
except:
    pass
# db = sqlite_utils.Database("wrc_2023_results.db")

#!rm wrc_2024_12_results.db
db = sqlite_utils.Database(_db_filepath)

In [179]:
# db["delme"].insert_all(df_fullcal.to_dict("records"))
# db["delme"].drop()

In [180]:
def db_add(table, df):
    if not df.empty:
        db[table].insert_all(df.to_dict("records"), alter=True)

db_add("fullcal", getFullCalendar())
db_add("resultscal", getResultsCalendar())

In [181]:
# We can add alter=True to dynamically modify tables

def drop_ephemera_cols(df, keep=None, inplace=True):
    """ We are working with unnormalised tables; this tidies them slightly."""
    if keep is None:
        keep = []
    else:
        keep = [keep] if isinstance(keep, str) else keep

    try:
        cols = ["driverCountry", "driverCountryImage",
                "coDriver", "coDriverId", "coDriverCountry","coDriverCountryImage",
                "teamId", "team/car", "teamLogo" ]
        dropcols = [col for col in cols if col in df.columns and col not in keep]
        df.drop(columns=dropcols, inplace=True)
    except:
        print("Can't drop cols...")
    if not inplace:
        return df

def splits_long(df):    
    # Identify the 'roundN' columns dynamically
    round_columns = [col for col in df.columns if col.startswith('S_round')]
    
    # Use pd.melt to transform the DataFrame into a long format
    long_df = pd.melt(
        df,
        #id_vars=[col for col in df.columns if col not in round_columns],  # Keep all columns except 'roundN' as identifiers
        id_vars=["pos", "carNo", "driverId", "driver", 	"splitPointId", "splitPointTimeId", "groupClass", "start","stageTime","diffFirst","eventId","rallyId","stageId"],
        value_vars=round_columns,  # Columns to unpivot
        var_name='round',          # Name of the new column for round labels
        value_name='time'          # Name of the new column for the round values
    )
    
    # Extract the N value from the 'round' column
    long_df['round_number'] = long_df['round'].str.extract('(\d+)$').astype(int)
    
    # Drop the original 'round' column if needed (optional)
    long_df = long_df.drop(columns=['round'])

    long_df['total_seconds'] = long_df['stageTime'].apply(time_to_seconds)

    return long_df

def stageresults(row, championshipId, eventId, rallyId):
    stageId = row["stageId"]
    if stageId=="FINAL":
        return
    try:
        _df_stageTimes = getStageTimes(eventId, rallyId, stageId)
        drop_ephemera_cols(_df_stageTimes)
        if 'event' in _df_stageTimes.columns:
            db_add("shakedown_stagetimes", _df_stageTimes)   
        else:
            db_add("stagetimes", _df_stageTimes)
    except:
        print(f"Couldn't get stagetimes for {eventId} {rallyId} {stageId}")
    try:
        _df_splitTimes = getSplitTimes(championshipId, eventId, rallyId, stageId)
        drop_ephemera_cols(_df_splitTimes)
        if 'event' in _df_splitTimes.columns:
            db_add("shakedown_split", _df_splitTimes)   
        else:
            # Create the new column 'round99'
            _df_splitTimes['round99'] = _df_splitTimes.apply(
                lambda row: row['stageTime'] if row.name == 0 else row['diffFirst'], axis=1
                )
            db_add("splittimes", _df_splitTimes)
            try:
                round_cols = [c for c in _df_splitTimes.columns if c.startswith("round")]
                for col in round_cols:
                    base_time = _df_splitTimes.loc[0, col]  # The base time is in the first row
                    # Apply the time deltas starting from the second row
                    _df_splitTimes[f"S_{col}"] = _df_splitTimes[col].apply(lambda x: time_to_seconds(x) if x == base_time else apply_time_delta(base_time, x))

                _df_long_splits = splits_long(_df_splitTimes)
                _df_long_splits['timeshift'] = _df_long_splits.groupby(["rallyId", "stageId", "driverId"])['time'].shift(periods=1, fill_value=0)
                _df_long_splits['duration'] = round(_df_long_splits['time'] - _df_long_splits['timeshift'],1)
                _df_long_splits.drop(columns=["timeshift"], inplace=True)
                db_add("splittimes_long", _df_long_splits)
            except:
                print(f"Couldn't make {eventId} {rallyId} {stageId} splits long")
    except:
          print(f"Couldn't get splittimes for {eventId} {rallyId} {stageId}")
    try:
        _df_Overall = getOverall(eventId, rallyId, stageId)
        if 'event' in _df_Overall.columns:
            db_add("shakedown_overall", _df_Overall)   
        else:
            db_add("overall", _df_Overall)
    except:
        print(f"Couldn't get overall for {eventId} {rallyId} {stageId}")


def rallyresults(row):
    eventId, rallyId = row[["eventId", "rallyId"]]
    db_add("retirements", drop_ephemera_cols(getRetirements(eventId), keep="team/car", inplace=False))
    db_add("penalties", drop_ephemera_cols(getPenalties(eventId), keep="team/car", inplace=False))
    db_add("startlist", getStartlist(eventId))
    db_add("itinerary", getItinerary(eventId))
    db_add("stagewinners", drop_ephemera_cols(getStageWinners(eventId), keep="team/car", inplace=False))

    df_stageDetails = getStageDetails(eventId, rallyId)
    db_add("stagedetails", df_stageDetails)
    df_stageDetails.apply(stageresults, axis=1, args=(championshipId, eventId, rallyId))
    #stageresults(df_stageDetails)


def rallyresults2(row):
    eventId, rallyId = row[["eventId", "rallyId"]]
    print(eventId, rallyId, row["startDateLocal"])

_ = df_fullcal.apply(rallyresults, axis=1)

Couldn't get splittimes for 459 490 8175


In [182]:
_ = df_fullcal.apply(rallyresults2, axis=1)

446 477 2024-01-24T16:30:00+01:00
447 478 2024-02-15T09:00:00+01:00
448 479 2024-03-27T10:00:00+03:00
449 480 2024-04-18T09:00:00+02:00
450 481 2024-05-09T08:00:00+01:00
452 483 2024-05-30T18:45:00+02:00
453 484 2024-06-27T10:00:00+02:00
454 485 2024-07-18T07:31:00+03:00
455 486 2024-07-31T19:00:00+03:00
456 487 2024-09-05T09:01:00+03:00
457 488 2024-09-26T08:31:00-03:00
458 489 2024-10-17T09:30:00+02:00
459 490 2024-11-21T09:00:00+09:00


In [192]:
df_fullcal.columns

Index(['id', 'guid', 'title', 'location', 'startDate', 'endDate', 'eventId',
       'rallyId', 'description', 'round', 'cvpSeriesLink', 'sponsor', 'images',
       'season', 'competition', 'country', 'asset', '__typename', 'type',
       'uid', 'seriesUid', 'releaseYear', 'availableOn', 'availableTill',
       'startDateLocal', 'endDateLocal', 'finishDate', 'championship',
       'championshipLogo'],
      dtype='object')

In [104]:
for row in db.query("select * from penalties LIMIT 3 "):
    print(row)

{'id': '3470', 'carNo': '#50', 'driverId': 'd2bd2b99-8877-5c1e-9823-0482cbdd9150', 'driver': 'Henk VOSSEN', 'team/car': 'Fiesta', 'teamName': 'Ford', 'eligibility': 'WRC2 (DCM/CCM)', 'groupClass': 'RC2', 'rallyId': '357', 'entryId': '36979', 'controlId': '17593', 'penaltyTime': '10.0', 'penaltyDuration': 'PT10S', 'penaltyDurationMs': '10000', 'reason': '1 MIN LATE', 'control': 'TC3'}
{'id': '3471', 'carNo': '#84', 'driverId': '42d8a1fc-8e31-5ea9-8c9f-1c1957f93dcf', 'driver': 'Nicolas RESSEGAIRE', 'team/car': 'Clio RS Line', 'teamName': 'Renault', 'eligibility': '', 'groupClass': 'RC5', 'rallyId': '357', 'entryId': '37012', 'controlId': '17597', 'penaltyTime': '10.0', 'penaltyDuration': 'PT10S', 'penaltyDurationMs': '10000', 'reason': 'FALSE START', 'control': 'SS4'}
{'id': '3472', 'carNo': '#80', 'driverId': '789de634-fdc6-5489-8f9e-d0cbb4fa5ee2', 'driver': 'Jérôme AYMARD', 'team/car': 'Clio RS', 'teamName': 'Renault', 'eligibility': '', 'groupClass': 'RC4', 'rallyId': '357', 'entryId'

In [97]:
for row in db.query("select * from splittimes_long LIMIT 3 "):
    print(row)

{'pos': '1', 'carNo': '#69', 'driverId': 'd8e4bbea-3af2-5486-9ad5-a445aaec573e', 'driver': 'Kalle ROVANPERÄ', 'splitPointId': '7252', 'splitPointTimeId': '256369', 'groupClass': 'RC1', 'start': '2023-01-19T20:05:00+01:00', 'stageTime': '10:30.3', 'diffFirst': '', 'eventId': '353', 'rallyId': '357', 'stageId': '4161', 'time': 267.6, 'round_number': 1, 'total_seconds': 630.3, 'duration': 267.6}
{'pos': '2', 'carNo': '#8', 'driverId': '6632e7ca-34bf-55b8-9cad-d060000fa794', 'driver': 'Ott TÄNAK', 'splitPointId': '7252', 'splitPointTimeId': '256370', 'groupClass': 'RC1', 'start': '2023-01-19T20:08:00+01:00', 'stageTime': '10:31.5', 'diffFirst': '+1.2', 'eventId': '353', 'rallyId': '357', 'stageId': '4161', 'time': 268.6, 'round_number': 1, 'total_seconds': 631.5, 'duration': 268.6}
{'pos': '3', 'carNo': '#11', 'driverId': 'c99a2a26-bd03-5153-aaa7-684d3acb5491', 'driver': 'Thierry NEUVILLE', 'splitPointId': '7252', 'splitPointTimeId': '256373', 'groupClass': 'RC1', 'start': '2023-01-19T20:1

In [99]:
for row in db.query("select * from splittimes_long WHERE carNo='#11' and stageId= '4161' "):
    print(row)

{'pos': '3', 'carNo': '#11', 'driverId': 'c99a2a26-bd03-5153-aaa7-684d3acb5491', 'driver': 'Thierry NEUVILLE', 'splitPointId': '7252', 'splitPointTimeId': '256373', 'groupClass': 'RC1', 'start': '2023-01-19T20:11:00+01:00', 'stageTime': '10:28.9', 'diffFirst': '-1.4', 'eventId': '353', 'rallyId': '357', 'stageId': '4161', 'time': 268.7, 'round_number': 1, 'total_seconds': 628.9, 'duration': 268.7}
{'pos': '3', 'carNo': '#11', 'driverId': 'c99a2a26-bd03-5153-aaa7-684d3acb5491', 'driver': 'Thierry NEUVILLE', 'splitPointId': '7252', 'splitPointTimeId': '256373', 'groupClass': 'RC1', 'start': '2023-01-19T20:11:00+01:00', 'stageTime': '10:28.9', 'diffFirst': '-1.4', 'eventId': '353', 'rallyId': '357', 'stageId': '4161', 'time': 465.1, 'round_number': 2, 'total_seconds': 628.9, 'duration': 196.4}
{'pos': '3', 'carNo': '#11', 'driverId': 'c99a2a26-bd03-5153-aaa7-684d3acb5491', 'driver': 'Thierry NEUVILLE', 'splitPointId': '7252', 'splitPointTimeId': '256373', 'groupClass': 'RC1', 'start': '20

In [242]:
for row in db.query("select total_seconds, sum(duration) from splittimes_long WHERE carNo='#11' and stageId= '6144' group by carNo, stageId "):
    print(row)

{'total_seconds': 741.2, 'sum(duration)': 741.2}


In [42]:
!ls -al wrc_2024_12_results.db

-rw-r--r--  1 tonyhirst  staff  3108864 27 Nov 00:07 wrc_2024_12_results.db


In [161]:
for row in db.query("select * from splittimes LIMIT 3 "):
    print(row)

{'id': '8a4b14c8-b0df-5020-a962-f9086ad42754', 'pos': '1', 'carNo': '#33', 'driverId': 'ae7329c9-79b3-5d96-886c-22cca6217764', 'driver': 'Elfyn EVANS', 'coDriverId': '53e56691-fe7c-5271-9dc5-8960df28b221', 'coDriver': 'Scott MARTIN', 'teamId': 'be461a0c-d1fd-5052-a69c-3fd94f8cf5f6', 'team/car': 'GR Yaris Rally1 HYBRID', 'teamName': 'Toyota', 'eligibility': 'M', 'splitPointId': '10146', 'splitPointTimeId': '348919', 'groupClass': 'RC1', 'start': '2024-01-25T20:35:00+01:00', 'stageTime': '12:12.9', 'diffFirst': '', 'round1': '3:59.4', 'round2': '6:47.6', 'round3': '10:24.9', 'round4': '11:51.7', 'eventId': '446', 'rallyId': '477', 'stageId': '6144', 'round5': None, 'round6': None, 'round7': None, 'round8': None, 'round9': None, 'round10': None, 'round11': None, 'round12': None, 'round13': None}
{'id': '6d968d85-e78a-5d69-bcef-7fd4a3d9d85d', 'pos': '2', 'carNo': '#11', 'driverId': 'c99a2a26-bd03-5153-aaa7-684d3acb5491', 'driver': 'Thierry NEUVILLE', 'coDriverId': 'b1b98699-0332-528a-8a80-

In [633]:
db.table_names()

['fullcal',
 'resultscal',
 'retirements',
 'penalties',
 'startlist',
 'itinerary',
 'stagedetails',
 'shakedown_stagetimes',
 'shakedown_split',
 'shakedown_overall',
 'stagetimes',
 'splittimes',
 'splittimes_long',
 'overall']